# This notebook is demonstrating (and testing) scenarios for generating requests to the AICI server (evolving API)

In [1]:
import sys
#sys.path.insert(0, 'c:\\users\\emrek\\source\\guidance\\prompt-plan\\promptlib\\')
sys.path.insert(0, '/workspaces/aici/promptlib')


In [2]:
from promptlib import PromptNode
from promptlib import set_model, append, gen, choose, begin, end
from promptlib.models import LLM, TransformersLLM
from promptlib.endpoints import AICI

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Here are examples of plans generated by promptlib

First, we show building a single path through a prompt tree.

In [3]:
llm = TransformersLLM("gpt2")

pn = PromptNode().set_model(llm)                                          \
    .append("Here is a generated prompt:", tag="prompt1").gen(max_tokens=10)             \
    .append("Here is another generated prompt. ").gen(max_tokens=10, ignore=["prompt1"])

aici_steps = pn.build_linear_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'Fixed': {'text': 'Here is a generated prompt:', 'tag': 'prompt1'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. '}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}, 'mask_tags': ['prompt1']}}]}


In [4]:
llm = TransformersLLM("gpt2")

pn0  = PromptNode()

pn = pn0.set_model(llm)                                          \
    .begin(id="block1") \
    .append("Here is a generated prompt. Resp:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. Resp:").gen(max_tokens=10)      \
    .end() \
    .append("Here is a final prompt. Resp:").gen(max_tokens=10)  
    

aici_steps = pn0.build_tree_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'block': {'steps': [{'Fixed': {'text': 'Here is a generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}], 'id': 'block1'}}, {'Fixed': {'text': 'Here is a final prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


The constraints we are imposing here have both a python code implementation and also return a regex that can impose the same constraint.

The general idea is that a constraint in python has to match some implementation of a constraint in the Rust/WASM AST-Runner.  If none exists, then we need to upload imperative code (in WASM) to the server to implement the constraint.

In [5]:
llm = TransformersLLM("gpt2")

from promptlib.constraints import DummyCharacterConstraint, OddEvenDigitConstraint

pn0  = PromptNode()

pn = pn0.set_model(llm)                                          \
    .begin(id="block1") \
    .constrain(DummyCharacterConstraint(['h','e','l','o'])) \
    .constrain(OddEvenDigitConstraint()) \
    .append("Here is a generated prompt. Resp:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. Resp:").gen(max_tokens=10)      \
    .end() \
    .append("Here is a final prompt. Resp:").gen(max_tokens=10)  
    

aici_steps = pn0.build_tree_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'block': {'steps': [{'constraint': {'type': 'regex', 'rx': '[h|e|l|o]+'}}, {'constraint': {'type': 'regex', 'rx': '^(?:[13579][02468])*[13579]|[02468]([13579]?|(?:[13579][02468])*)$'}}, {'Fixed': {'text': 'Here is a generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}], 'id': 'block1'}}, {'Fixed': {'text': 'Here is a final prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


Here's an example of actually uploading a program and plan to the server

In [3]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please answer the following questions:\n Q: Who is the president of the USA?\n A:").gen(max_tokens=10)             \
    .append("\nQ: And who is the vice president?\n A:", tag="allcaps").gen(max_tokens=20) \
    .append("\nPlease give a url with evidence\n http://").gen(max_tokens=20, ignore=["allcaps"])

endpoint.run(pn)

upload module... 2688kB -> 7999kB id:6abb139f
{'model': '', 'prompt': '', 'max_tokens': 200, 'n': 1, 'temperature': 0, 'stream': True, 'aici_module': '6abb139fdd3b596815e342dc3bb124a8cc4fbc560a75f3e94965f0e8954957e1', 'aici_arg': {'steps': [{'Fixed': {'text': 'Please answer the following questions:\n Q: Who is the president of the USA?\n A:'}}, {'Gen': {'max_tokens': 10, 'genargs': {}}}, {'Fixed': {'text': '\nQ: And who is the vice president?\n A:', 'tag': 'allcaps'}}, {'Gen': {'max_tokens': 20, 'genargs': {}}}, {'Fixed': {'text': '\nPlease give a url with evidence\n http://'}}, {'Gen': {'max_tokens': 20, 'genargs': {}, 'mask_tags': ['allcaps']}}]}}
Please answer the following questions:
 Q: Who is the president of the USA?
 A: The president of the USA is Donald Trump.
Q: And who is the vice president?
 A: The vice president of the USA is Mike Pence.
Please give a url with evidence
 http://www.whitehouse.gov/

 Q: Who is the prime minister of the UK?


In [6]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append(" Please answer the following questions") \
    .append("(And answer in ALL CAPS):", tag="allcaps") \
    .append("\nQ: Why did Einstein say he liked to play violin?\n A:").gen(max_tokens=30, ignore=["allcaps"])             \
    .append("\nQ: Who is the president of the USA?\n A:").gen(max_tokens=30, ignore=["allcaps"])             \
    .append("\nQ: And who is the vice president?\n A:").gen(max_tokens=20) \
    .append("\nPlease give a url with evidence\n http://").gen(max_tokens=20)

aici_steps = endpoint.build_tree_plan()
print(aici_steps)

endpoint.run(pn)

upload module... 2688kB -> 7999kB id:6abb139f
{'steps': [{'Fixed': {'text': ' Please answer the following questions'}}, {'Fixed': {'text': '(And answer in ALL CAPS):', 'tag': 'allcaps'}}, {'Fixed': {'text': '\nQ: Why did Einstein say he liked to play violin?\n A:'}}, {'Gen': {'max_tokens': 30, 'genargs': {}, 'mask_tags': ['allcaps']}}, {'Fixed': {'text': '\nQ: Who is the president of the USA?\n A:'}}, {'Gen': {'max_tokens': 30, 'genargs': {}, 'mask_tags': ['allcaps']}}, {'Fixed': {'text': '\nQ: And who is the vice president?\n A:'}}, {'Gen': {'max_tokens': 20, 'genargs': {}}}, {'Fixed': {'text': '\nPlease give a url with evidence\n http://'}}, {'Gen': {'max_tokens': 20, 'genargs': {}}}]}
{'model': '', 'prompt': '', 'max_tokens': 200, 'n': 1, 'temperature': 0, 'stream': True, 'aici_module': '6abb139fdd3b596815e342dc3bb124a8cc4fbc560a75f3e94965f0e8954957e1', 'aici_arg': {'steps': [{'Fixed': {'text': ' Please answer the following questions'}}, {'Fixed': {'text': '(And answer in ALL CAPS):

In [30]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please answer the following questions") \
    .append("\nQ: Why did Einstein say he liked to play violin?\n") \
    .append("\n(And always answer in ALL CAPS)", tag="allcaps") \
    .append("\nA:") \
    .gen(max_tokens=30) #, ignore=["allcaps"])

#aici_steps = endpoint.build_tree_plan()
#print(aici_steps)

endpoint.run(pn)

upload module... 2688kB -> 7999kB id:6abb139f
{'model': '', 'prompt': '', 'max_tokens': 200, 'n': 1, 'temperature': 0, 'stream': True, 'aici_module': '6abb139fdd3b596815e342dc3bb124a8cc4fbc560a75f3e94965f0e8954957e1', 'aici_arg': {'steps': [{'Fixed': {'text': 'Please answer the following questions'}}, {'Fixed': {'text': '\nQ: Why did Einstein say he liked to play violin?\n'}}, {'Fixed': {'text': '\n(And always answer in ALL CAPS)', 'tag': 'allcaps'}}, {'Fixed': {'text': '\nA:'}}, {'Gen': {'max_tokens': 30, 'genargs': {}}}]}}
Please answer the following questions
Q: Why did Einstein say he liked to play violin?

(And always answer in ALL CAPS)
A: BECAUSE HE WAS A VIOLINIST

Comment: I'm voting to close this question as off-topic because it is


In [33]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please repeat the following list in order:") \
    .append("\nApple") \
    .append("\nCherries", tag="selected") \
    .append("\nGrapes", tag="selected") \
    .append("\nStrawberries") \
    .append("\nBananas", tag="selected") \
    .append("\n\nOk now please repeat the list:\n") \
    .gen(max_tokens=30, ignore=["selected"])

#aici_steps = endpoint.build_tree_plan()
#print(aici_steps)

endpoint.run(pn)

upload module... 2688kB -> 7999kB id:6abb139f
{'model': '', 'prompt': '', 'max_tokens': 200, 'n': 1, 'temperature': 0, 'stream': True, 'aici_module': '6abb139fdd3b596815e342dc3bb124a8cc4fbc560a75f3e94965f0e8954957e1', 'aici_arg': {'steps': [{'Fixed': {'text': 'Please repeat the following list in order:'}}, {'Fixed': {'text': '\nApple'}}, {'Fixed': {'text': '\nCherries', 'tag': 'selected'}}, {'Fixed': {'text': '\nGrapes', 'tag': 'selected'}}, {'Fixed': {'text': '\nStrawberries'}}, {'Fixed': {'text': '\nBananas', 'tag': 'selected'}}, {'Fixed': {'text': '\n\nOk now please repeat the list:\n'}}, {'Gen': {'max_tokens': 30, 'genargs': {}}}]}}
Please repeat the following list in order:
Apple
Cherries
Grapes
Strawberries
Bananas

Ok now please repeat the list:
Apple
Cherries
Grapes
Strawberries
Bananas

Please repeat the following list in order:

